# Project 2 Extract Transform and Load

Process CSV File (Player data, Team data, Match Data)
Process URL Scrapping (Stadium data from wikipedia)
Process API Data (Statdium Coordinates and Weather on the day)

In [1]:
import pandas as pd
import numpy as np
import requests
import pymongo
from datetime import timezone
from sqlalchemy import create_engine

from my_etl_utilities import convert_to_json
from api_keys import weather_api_key
from api_keys import postgres_user
from api_keys import postgres_pwd
from api_keys import postgres_host
from api_keys import postgres_port

# Extract and Transform CSV into DataFrame

## CSV Data Scrapping

### Team Data (An example of simple transformation)
* Read CSV
* Create a sub-frame
* Rename and transform
* Apply proper datatypes

In [2]:
#Extract
team_file = "Resources/TeamData.csv"
team_data_df = pd.read_csv(team_file)

#Transform
team_req_cols = ["team", "players_used", "avg_age","possession","games","minutes","cards_yellow","cards_red","goals_assists_pens_per90"]
team_df = team_data_df[team_req_cols].copy()

#Rename columns and organize data
team_df = team_df.rename(columns={"goals_assists_pens_per90":"performance"})
team_df.sort_values(by=['performance'],ascending=False,inplace=True)

# #Set proper data types
data_types_dict = {'team': str,'players_used': str,'avg_age':float,'possession':int,'minutes':int}
team_df = team_df.astype(data_types_dict)

#Set index
#team_df = team_df.set_index("team")
team_df.head()

,team,players_used,avg_age,possession,games,minutes,cards_yellow,cards_red,performance
10,England,20,27.0,62,5,450,1,0,4.60
21,Portugal,24,27.8,60,5,450,6,0,4.00
11,France,24,27.3,52,6,540,5,0,4.00
19,Netherlands,21,27.6,53,5,480,12,1,3.38
12,Germany,20,28.1,59,3,270,3,0,3.33


### Player Data (CSV) (An example of string processing on data frame columns )
* Read CSV
* Create a sub-frame
* Rename and transform age by splitting string of a dataframe column
* Filter records by performance number
* Apply proper datatypes

In [3]:
#Extract
players_file = "Resources/PlayerStats.csv"
players_data_df = pd.read_csv(players_file)

#Transform
#"goals_assists_pens_per90": "Goals plus Assists minus Penalty Kicks made per 90 minutes. Minimum 30 minutes played per squad game to qualify as a leader"
players_req_cols = ["player", "position", "team","age","goals_assists_pens_per90"]
players_df = players_data_df[players_req_cols].copy()

#Rename columns and organize data
players_df = players_df.rename(columns={"goals_assists_pens_per90":"performance"})
players_df.sort_values(by=['performance'],ascending=False,inplace=True)

#Transform the age column
players_df['age'] = players_df.age.str.split("-").str[0]

#Set proper data types
data_types_dict = {'player': str,'position': str,'team':str,'age':int,'performance':float}
players_df = players_df.astype(data_types_dict)
players_df.dtypes

#Apply criteria to filter data 
players_df = players_df[players_df.performance < 4.0]

#Set index
#players_df = players_df.set_index("player")
players_df.head()


,player,position,team,age,performance
477,Niclas Füllkrug,FW,Germany,29,3.91
503,Paik Seung-ho,MF,Korea Republic,25,3.46
643,Wout Weghorst,FW,Netherlands,30,2.81
446,Mislav Oršić,FW,Croatia,29,2.47
227,Gonçalo Ramos,FW,Portugal,21,2.37


### Match Data (An example of transformation of data type)
* Read CSV
* Create a sub-frame
* Rename and transform - convert from string data type to datetime 
* Apply proper datatypes

In [4]:
#Extract
match_file = "Resources/MatchData.csv"
match_data_df = pd.read_csv(match_file)

#Transform
match_req_cols = ["match","dayofweek","match_time","home_team","away_team","score","attendance","venue"]
match_df = match_data_df[match_req_cols].copy()

#Set proper data types
data_types_dict = {'match': str,'dayofweek': str,'match_time':str,'home_team':str,'away_team':str,'score':str,'attendance':int,'venue':str}
match_df = match_df.astype(data_types_dict)
match_df['dayofweek'] = match_df['dayofweek'].str.slice(start=0,stop=3)

#convert to date and time
# match_df['match_time'] = pd.to_datetime(match_df['match_time'])
# match_df['utc_time'] = match_df['match_time']
# match_df['utc_time'] = match_df['utc_time'].astype(np.int64)
# match_df.dtypes

# #Set index
#match_df = match_df.set_index("match")
match_df.head()

,match,dayofweek,match_time,home_team,away_team,score,attendance,venue
0,1,Sun,11/20/2022 19:00,Qatar,Ecuador,0–2,67372,Al Bayt Stadium
1,2,Mon,11/21/2022 16:00,England,IR Iran,6–2,45334,Khalifa International Stadium
2,3,Mon,11/21/2022 19:00,Senegal,Netherlands,0–2,41721,Al Thumama Stadium
3,4,Mon,11/21/2022 22:00,United States,Wales,1–1,43418,Ahmed bin Ali Stadium
4,5,Tue,11/22/2022 13:00,Argentina,Saudi Arabia,1–2,88012,Lusail Iconic Stadium


## Panda Web Scraping
* Set URL for source
* Call Pandas Web scraping method
* Filter tables and transform
* Apply proper data types

Note:
The "Lisail Stadium" from the wikipedia site did not match with the description in Match file
The match file had "Lisail Iconic Stadium". The statdium data from wikipedia was adjusted to match with csv data


In [5]:
url = 'https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Venues'
stadium_data = pd.read_html(url)
stadium_v1_df = stadium_data[4]

#find and replace 'Lusail Stadium' with 'Lusail Iconic Stadium'
stadium_v1_df.loc[(stadium_v1_df.Stadium == 'Lusail Stadium'), 'Stadium'] = 'Lusail Iconic Stadium'

#Clean up the capacity column
#Step 1: 88,966[82][83] -> 88,966
#Step 2: replace "," with null 
#Step 3: convert the value into numeric
stadium_v1_df['Capacity'] = pd.to_numeric((stadium_v1_df['Capacity'].str.split("[").str[0]).str.replace(",",""))

#Set proper data types
data_types_dict = {'Stadium': str,'City': str,'Capacity':int}
stadium_v1_df = stadium_v1_df.astype(data_types_dict)

# stadium_v1_df.set_index("Stadium",inplace=True)
stadium_v1_df

,City,Stadium,Capacity
0,Lusail,Lusail Iconic Stadium,88966
1,Al Khor,Al Bayt Stadium,68895
2,Al Rayyan,Khalifa International Stadium,45857
3,Al Rayyan,Ahmad bin Ali Stadium,45032
4,Al Rayyan,Education City Stadium,44667
5,Doha,Al Thumama Stadium,44400
6,Doha,Stadium 974,44089
7,Al Wakrah,Al Janoub Stadium,44325


## API Data Extraction

In [6]:
#Get Stadium Data with GPS info
stadium_file = "D:/UPENN/Project2/Project2-ETL/Resources/Stadium.csv"
stadium_v2_df = pd.read_csv(stadium_file)
stadium_v2_df.drop(columns='City',inplace=True)
stadium_df = pd.merge(stadium_v1_df,stadium_v2_df,how="inner",on="Stadium")
stadium_df

,City,Stadium,Capacity,Lat,Lon
0,Lusail,Lusail Iconic Stadium,88966,25.422152,51.490266
1,Al Khor,Al Bayt Stadium,68895,25.653061,51.487936
2,Al Rayyan,Khalifa International Stadium,45857,25.264134,51.448493
3,Al Rayyan,Ahmad bin Ali Stadium,45032,25.329797,51.342447
4,Al Rayyan,Education City Stadium,44667,25.311527,51.424203
5,Doha,Al Thumama Stadium,44400,25.235689,51.532532
6,Doha,Stadium 974,44089,25.289761,51.566629
7,Al Wakrah,Al Janoub Stadium,44325,25.160004,51.574365


In [15]:
#historical_url = f'https://history.openweathermap.org/data/3.0/history/timemachine?appid={weather_api_key}'
url = f'https://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}'

match_df["temp"] = ''
match_df["feels_like"] = ''
match_df["temp_min"] = ''
match_df["temp_max"] = ''
match_weather_df=pd.DataFrame()

#For each stadium get the weather info the Match data
for index, stadium in stadium_df.iterrows():
    l_stadium = stadium['Stadium']
    lon = stadium['Lon']
    lat = stadium['Lat']    
    weather_url = f'{url}&lat={lat}&lon={lon}'
    response = requests.get(weather_url)
    weather_json = response.json()

    #Extract data from JSON
    temp=weather_json['main']['temp']
    feels_like=weather_json['main']['feels_like']
    temp_min=weather_json['main']['temp_min']
    temp_max=weather_json['main']['temp_max']

    #Locate and append the values to the match  dataframe
    match_sub_df = match_df.loc[match_df['venue'] == stadium['Stadium']]
    match_sub_df = match_sub_df.set_index(['dayofweek',"match_time"])
    match_sub_df.loc[:,("temp")]=temp
    match_sub_df.loc[:,("feels_like")]=feels_like
    match_sub_df.loc[:,("temp_min")]=temp_min
    match_sub_df.loc[:,("temp_max")]=temp_max
    match_weather_df = pd.concat([match_weather_df,match_sub_df])

match_weather_df = match_weather_df.reset_index()
match_weather_df.head()

,dayofweek,match_time,match,home_team,away_team,score,attendance,venue,temp,feels_like,temp_min,temp_max
0,Tue,11/22/2022 13:00,5,Argentina,Saudi Arabia,1–2,88012,Lusail Iconic Stadium,295.0,294.91,289.33,296.1
1,Thu,11/24/2022 22:00,16,Brazil,Serbia,2–0,88103,Lusail Iconic Stadium,295.0,294.91,289.33,296.1
2,Sat,11/26/2022 22:00,24,Argentina,Mexico,2–0,88966,Lusail Iconic Stadium,295.0,294.91,289.33,296.1
3,Mon,11/28/2022 22:00,32,Portugal,Uruguay,2–0,88668,Lusail Iconic Stadium,295.0,294.91,289.33,296.1
4,Wed,11/30/2022 22:00,39,Saudi Arabia,Mexico,1–2,84985,Lusail Iconic Stadium,295.0,294.91,289.33,296.1


# Mongo DB - Persist Stadium data in MongoDB with stadium info and unstructured data

In [17]:
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.FIFA_2022_DB
collections = db.Stadiums
collections.delete_many({}) #Destroy the collection

#Convert dataframe to dictionary
stadium_list = stadium_df.to_dict('records')
collections.insert_many(stadium_list)

# #update in chunks
# chunk_size=10
# list_len = len(stadium_list)
# ran=range(list_len)
# steps=ran[chunk_size::chunk_size]
# i=0
# for j in steps:
#     collections.insert_many(stadium_list[i:j])
#     i=j


![MongoDB Stadium List](../screenshots/mongo_stadium_list.png)

### Connect to local postgres database

In [18]:
protocol = 'postgresql'
username = postgres_user
password = postgres_pwd
host = postgres_host
port = postgres_port
database_name = 'fifa_2022_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)


### Check for tables

In [19]:
engine.table_names()

C:\Users\anand\AppData\Local\Temp\ipykernel_12776\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Players', 'Match', 'Teams']

### Use pandas to load converted DataFrame into database

In [20]:
engine.execution_options(autocommit=True)
# engine.execute(statement='Truncate Table Match')
match_weather_df.to_sql(name='Match', con=engine, if_exists='append', index=False)

55

In [22]:
#engine.execute(statement='Truncate Table public.Players')
players_df.to_sql(name='Players', con=engine, if_exists='append',index=False)

673

In [23]:
#engine.execute(statement='Truncate Table public.Teams')
team_df.to_sql(name='Teams',con=engine,if_exists='append',index=False)

32

### Using Pandas read the top rows from the database table

In [24]:
pd.read_sql_query('SELECT * FROM public."Match" ORDER BY match ASC ', con=engine).head()

,match,dayofweek,match_time,home_team,away_team,score,attendance,venue,temp,feels_like,temp_min,temp_max
0,1,Sun,11/20/2022 19:00,Qatar,Ecuador,0–2,67372,Al Bayt Stadium,294.18,293.98,294.18,296.12
1,2,Mon,11/21/2022 16:00,England,IR Iran,6–2,45334,Khalifa International Stadium,294.42,294.17,289.23,296.00
2,3,Mon,11/21/2022 19:00,Senegal,Netherlands,0–2,41721,Al Thumama Stadium,294.86,294.63,289.28,296.05
3,5,Tue,11/22/2022 13:00,Argentina,Saudi Arabia,1–2,88012,Lusail Iconic Stadium,295.00,294.91,289.33,296.10
4,6,Tue,11/22/2022 16:00,Denmark,Tunisia,0–0,42925,Education City Stadium,294.45,294.20,289.25,296.03


In [26]:
pd.read_sql_query('SELECT * FROM public."Players" ORDER BY plid ASC ', con=engine).head()

,plid,player,position,team,performance,age
0,2,Niclas Füllkrug,FW,Germany,3.91,29
1,3,Paik Seung-ho,MF,Korea Republic,3.46,25
2,4,Wout Weghorst,FW,Netherlands,2.81,30
3,5,Mislav Oršić,FW,Croatia,2.47,29
4,6,Gonçalo Ramos,FW,Portugal,2.37,21


In [27]:
pd.read_sql_query('SELECT * FROM public."Teams" ORDER BY teamid ASC ', con=engine).head()

,teamid,team,players_used,avg_age,possession,games,minutes,cards_yellow,cards_red,performance
0,2,England,20,27.0,62,5,450.0,1,0,4.60
1,3,Portugal,24,27.8,60,5,450.0,6,0,4.00
2,4,France,24,27.3,52,6,540.0,5,0,4.00
3,5,Netherlands,21,27.6,53,5,480.0,12,1,3.38
4,6,Germany,20,28.1,59,3,270.0,3,0,3.33
